# Plotting Tests

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from fau_colors import cmaps, register_fausans_font

from pepbench.datasets import EmpkinsDataset, GuardianDataset
from pepbench.io import load_challenge_results_from_folder

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
deploy_type = "local"
config_dict = json.load(Path("./config.json").open(encoding="utf-8"))

guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])

In [ ]:
results_path = Path("../experiments/pep_algorithm_benchmarking/results")

In [ ]:
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
dataset_guardian

In [ ]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
dataset_empkins

In [ ]:
participant = "VP_001"
condition = "tsst"
phase = "Pause_1"

In [ ]:
datapoint = dataset_empkins.get_subset(participant=participant, condition=condition, phase=phase)

In [ ]:
from pepbench.plotting import (
    plot_signals,
    plot_signals_from_challenge_results,
    plot_signals_with_reference_labels,
    plot_signals_with_reference_pep,
)

In [ ]:
fig, axs = plot_signals(
    datapoint,
    collapse=False,
    legend_outside=True,
    normalize_time=True,
    heartbeat_subset=(0, 4),
    legend_orientation="horizontal",
)

In [ ]:
fig, axs = plot_signals_with_reference_labels(
    datapoint,
    collapse=True,
    normalize_time=True,
    heartbeat_subset=(1, 4),
    legend_outside=True,
    plot_artefacts=True,
    legend_orientation="horizontal",
)

In [ ]:
fig, axs = plot_signals_with_reference_pep(
    datapoint,
    normalize_time=True,
    heartbeat_subset=(0, 4),
    legend_outside=True,
    legend_orientation="horizontal",
    pep_hatch="//",
)

In [ ]:
results_type = "empkins_dataset_both_algorithms"

folder_path = results_path.joinpath(results_type)
folder_path

In [ ]:
results_agg_mean_std, results_agg_total, results_single, results_per_sample = load_challenge_results_from_folder(
    folder_path,
    index_cols_single=["participant", "condition", "phase"],
    index_cols_per_sample=["participant", "condition", "phase"],
)

In [ ]:
results_per_sample

In [ ]:
pipeline_result = results_per_sample.loc[("r-peak-diff-40-ms", "straight-line", "autoregression")]
pipeline_result = pipeline_result.xs((participant, condition, phase), level=["participant", "condition", "phase"])

In [ ]:
fig, ax = plot_signals_from_challenge_results(
    datapoint,
    pipeline_result,
    normalize_time=True,
    heartbeat_subset=(0, 2),
    collapse=True,
    add_pep=True,
    legend_outside=True,
    legend_orientation="horizontal",
)

In [ ]:
from pepbench.plotting.algorithms import (
    plot_b_point_extraction_arbol2017,
    plot_b_point_extraction_debski1993,
    plot_b_point_extraction_drost2022,
    plot_b_point_extraction_forouzanfar2018,
    plot_q_peak_extraction_neurokit_dwt,
    plot_q_wave_onset_extraction_van_lien_2013,
)

In [ ]:
fig, ax = plot_q_peak_extraction_neurokit_dwt(datapoint, heartbeat_subset=(0, 8), normalize_time=True)

In [ ]:
fig, ax = plot_q_wave_onset_extraction_van_lien_2013(
    datapoint, heartbeat_subset=(0, 2), algo_params={"time_interval_ms": 40}, normalize_time=True
)

In [ ]:
fig, ax = plot_q_wave_onset_extraction_van_lien_2013(
    datapoint, heartbeat_subset=(0, 2), algo_params={"time_interval_ms": 40}, normalize_time=True
)

In [ ]:
fig, axs = plot_b_point_extraction_debski1993(datapoint, heartbeat_subset=(0, 2), normalize_time=True, algo_params=None)

In [ ]:
fig, axs = plot_b_point_extraction_arbol2017(datapoint, heartbeat_subset=(0, 2), normalize_time=True, algo_params=None)

In [ ]:
fig, axs = plot_b_point_extraction_drost2022(datapoint, heartbeat_subset=(0, 2), normalize_time=True, algo_params=None)

In [ ]:
fig, axs = plot_b_point_extraction_forouzanfar2018(
    datapoint, heartbeat_subset=(10, 12), normalize_time=True, algo_params=None
)

In [ ]:
test_path = Path("/Users/richer/Documents/PhD/Projects/BioPsyKit/tests/test_data/pep")

ecg_data = pd.read_csv(test_path.joinpath("pep_test_ecg.csv"), index_col=0)
ecg_data.index = pd.to_datetime(ecg_data.index)

icg_data = pd.read_csv(test_path.joinpath("pep_test_icg.csv"), index_col=0)
icg_data.index = pd.to_datetime(icg_data.index)

heartbeats = pd.read_csv(test_path.joinpath("pep_test_heartbeat_reference_variable_length.csv"), index_col=0)

q_wave_onsets = pd.read_csv(test_path.joinpath("pep_test_q_wave_onset_reference_neurokit_dwt.csv"), index_col=0)
q_wave_onsets = q_wave_onsets.dropna(subset="q_wave_onset_sample").convert_dtypes(infer_objects=True)

c_points = pd.read_csv(test_path.joinpath("pep_test_c_point_reference_scipy_findpeaks.csv"), index_col=0)

In [ ]:
fig, ax = plt.subplots()
ecg_data.plot(ax=ax)

ax.scatter(
    ecg_data.index[q_wave_onsets["q_wave_onset_sample"]],
    ecg_data.iloc[q_wave_onsets["q_wave_onset_sample"]],
    color=cmaps.tech[0],
    zorder=3,
)

ax.vlines(
    x=ecg_data.index[heartbeats["start_sample"]],
    ymin=0,
    ymax=1,
    zorder=0,
    colors=cmaps.tech[1],
    transform=ax.get_xaxis_transform(),
    ls="--",
)

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
icg_data.plot(ax=ax)

ax.scatter(
    icg_data.index[c_points["c_point_sample"]], icg_data.iloc[c_points["c_point_sample"]], color=cmaps.tech[0], zorder=3
)

fig.tight_layout()